In [1]:
import torch
import torch.nn as nn

/workspaces/data-mining/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [3]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [4]:
class LayerNorm(nn.Module):
    def __init__(self, emb_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emb_dim))
        self.shift = nn.Parameter(torch.zeros(emb_dim))

    def forward(self, x):
        mean = x.mean(dim=-1, keepdim=True)
        var = x.var(dim=-1, keepdim=True, unbiased=False)
        norm_x = (x - mean) / torch.sqrt(var + self.eps)
        return self.scale * norm_x + self.shift

In [5]:
class GELU(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(cfg["emb_dim"], 4 * cfg["emb_dim"]),
            GELU(),
            nn.Linear(4 * cfg["emb_dim"], cfg["emb_dim"]),
        )

    def forward(self, x):
        return self.layers(x)

In [6]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # As in `CausalAttention`, for inputs where `num_tokens` exceeds `context_length`, 
        # this will result in errors in the mask creation further below. 
        # In practice, this is not a problem since the LLM (chapters 4-7) ensures that inputs  
        # do not exceed `context_length` before reaching this forward method.

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [7]:
class TransformerBlock( nn.Module ):
    def __init__( self, cfg ):
        super().__init__()
        self.norm1 = LayerNorm( cfg["emb_dim"] )
        self.att = MultiHeadAttention( 
            d_in = cfg["emb_dim"], 
            d_out = cfg["emb_dim"], 
            context_length = cfg["context_length"], 
            dropout = cfg["drop_rate"], 
            num_heads = cfg["n_heads"] 
        )
        self.dropout = nn.Dropout( cfg["drop_rate"] )
        self.norm2 = LayerNorm( cfg["emb_dim"] )
        self.feedforward = FeedForward( cfg )
        
    def forward( self, x ):
        
        # the attention chunk in this transformer:
        shortcut = x 
        x = self.norm1( x )
        x = self.att( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        # the feed forward chunk:
        shortcut = x
        x = self.norm2( x ) 
        x = self.feedforward( x )
        x = self.dropout( x )
        x = x + shortcut 
        
        return x 

In [8]:
# instantiate TransformerBlock
block = TransformerBlock( GPT_CONFIG_124M )

In [9]:
input = torch.randn( 3, 4, 768 )

In [10]:
input

tensor([[[ 0.5365,  0.6304,  1.4286,  ..., -0.8589,  1.0146,  0.1822],
         [-0.1571,  0.7575,  0.3223,  ...,  0.1365, -2.3871, -0.9723],
         [-0.2004,  0.6843,  0.1361,  ...,  0.7054,  0.3981,  0.4207],
         [ 0.4324,  0.0907,  0.7021,  ..., -1.4437, -0.9341,  0.0220]],

        [[ 0.9855,  1.8203, -0.7292,  ...,  1.4249, -0.6743, -1.1458],
         [ 0.0839,  1.5041, -0.2549,  ...,  0.7667,  1.2359,  0.0746],
         [ 1.2848, -0.2965, -0.9341,  ...,  0.6674,  1.0094, -1.3318],
         [-0.3413,  0.0746, -1.8397,  ...,  1.3805, -0.6887,  0.2890]],

        [[ 0.3321,  0.3754,  1.4693,  ..., -0.2802,  0.3456, -0.0243],
         [-0.0248, -1.0027,  0.5380,  ..., -1.3649, -0.6335,  0.6071],
         [ 0.0454,  1.0774, -1.0968,  ...,  0.4678, -0.1130, -0.4069],
         [-0.3369,  0.4631,  1.5244,  ..., -0.9235,  0.9033,  0.9213]]])

In [11]:
output = block( input )

In [12]:
output.shape

torch.Size([3, 4, 768])

In [13]:
output

tensor([[[-8.7391e-02,  1.0003e+00,  1.4089e+00,  ..., -3.9943e-01,
           1.5810e+00,  3.5092e-01],
         [-3.6049e-01,  1.2388e+00,  3.9761e-03,  ..., -1.6467e-02,
          -2.2170e+00, -7.6890e-01],
         [-9.1071e-01,  8.1519e-01,  4.7745e-01,  ...,  8.6162e-01,
           4.2637e-01,  1.9251e-01],
         [ 1.3512e-01,  4.6240e-01,  7.0848e-01,  ..., -1.2576e+00,
          -9.3351e-01,  1.3931e-01]],

        [[ 7.1162e-01,  1.0770e+00, -1.2297e+00,  ...,  9.7028e-01,
          -7.0450e-01, -1.6460e+00],
         [-1.1584e-01,  1.4105e+00, -4.4189e-01,  ...,  4.1896e-02,
           9.7186e-01,  2.7600e-02],
         [ 1.8147e+00, -6.7550e-04, -5.7534e-01,  ...,  1.8731e-01,
           1.3144e+00, -1.5224e+00],
         [-3.7988e-01,  7.0921e-01, -1.5543e+00,  ...,  1.0958e+00,
          -1.3154e+00,  2.7247e-01]],

        [[-2.8747e-01,  2.3828e-01,  1.1983e+00,  ..., -5.2584e-01,
           3.0885e-01, -1.4729e-01],
         [-4.3744e-01, -1.0804e+00,  1.0772e+00,  .